In [ ]:
# @title ColabUniversalDownloader

import json
import os
import re
import sys
import threading
import time
import uuid
from pathlib import Path
from typing import Any, Callable, Dict, Iterable, List, Optional, Tuple

from IPython.display import HTML
from IPython.display import display
import ipywidgets as widgets  # type: ignore

REPO_URL = "https://github.com/MattyMroz/ColabUniversalDownloader.git"
REPO_DIR = "/content/ColabUniversalDownloader"
REQ_FILE_LOCAL = "requirements.txt"
SESSION_ID = time.strftime("%Y%m%d-%H%M%S") + "-" + uuid.uuid4().hex[:6]

IS_COLAB = ("COLAB_GPU" in os.environ) or ("/content/" in os.getcwd())
if IS_COLAB:
    BASE_TMP = f"/content/tmp_ColabUniversalDownloader/{SESSION_ID}"
else:
    BASE_TMP = str(Path(os.getcwd()) / "tmp_ColabUniversalDownloader" / SESSION_ID)
Path(BASE_TMP).mkdir(parents=True, exist_ok=True)


def _run_quiet(cmd: List[str]) -> int:
    try:
        import subprocess
        return subprocess.run(cmd, check=False, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL).returncode
    except Exception:
        return 1


def ensure_repo_and_requirements() -> None:
    try:
        if IS_COLAB:
            if not Path(REPO_DIR, "utils").exists():
                _run_quiet(["git", "clone", REPO_URL, REPO_DIR])
            os.chdir(REPO_DIR)
        elif Path("utils").exists():
            pass
        else:
            _run_quiet(["git", "clone", REPO_URL])
            if Path("ColabUniversalDownloader", "utils").exists():
                os.chdir("ColabUniversalDownloader")
        req = Path(REQ_FILE_LOCAL)
        if req.exists():
            _run_quiet([sys.executable, "-m", "pip", "install", "-q", "-r", str(req)])
    except Exception:
        pass


ensure_repo_and_requirements()
if IS_COLAB and REPO_DIR not in sys.path:
    sys.path.append(REPO_DIR)
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

from utils.google_drive import GoogleDriveManager  # type: ignore
from utils.mega import MegaDownloader  # type: ignore
from utils.pixeldrain import PixelDrainDownloader  # type: ignore

PROGRESS_PANEL = widgets.HTML(value="")
RESULTS_PANEL = widgets.HTML(value="")
HIDDEN_LOG = widgets.Output()
HIDDEN_LOG.layout.display = 'none'

UI_STATE: Dict[str, Any] = {
    "widget": None,
    "results": [],
    "gdrive": None,
    "pixeldrain": None,
    "mega": None,
    "folder_web": "",
    "drive_ctx": {},
    "delete_scheduler": {"thread": None, "folder_id": None, "delete_at": 0.0, "lock": threading.Lock()},
}

PIXEL_RE = re.compile(r"https?://pixeldrain\.com/(u|l)/([A-Za-z0-9]+)")
MEGA_FILE_RE = re.compile(r"https?://mega\.nz/file/[\w-]+#[\w-]+", re.IGNORECASE)
MEGA_FOLDER_RE = re.compile(r"https?://mega\.nz/folder/[\w-]+#[\w-]+", re.IGNORECASE)


def _html_escape(s: str) -> str:
    return (
        s.replace("&", "&amp;")
        .replace("<", "&lt;")
        .replace(">", "&gt;")
        .replace("\"", "&quot;")
        .replace("'", "&#39;")
    )


def _set_progress_panel(html_snippet: str) -> None:
    if html_snippet:
        PROGRESS_PANEL.value = (
            "<div class='cud-results'><div class='cud-results-title'>Postęp</div>" + html_snippet + "</div>"
        )
    else:
        PROGRESS_PANEL.value = ""


def parse_links(text: str) -> List[Dict[str, str]]:
    tasks: List[Dict[str, str]] = []
    for token in re.split(r"[\s,]+", text.strip()):
        if not token:
            continue
        if PIXEL_RE.search(token):
            tasks.append({"type": "pixeldrain", "url": token})
        elif MEGA_FOLDER_RE.search(token):
            tasks.append({"type": "mega_folder", "url": token})
        elif MEGA_FILE_RE.search(token):
            tasks.append({"type": "mega_file", "url": token})
        else:
            tasks.append({"type": "unknown", "url": token})
    return tasks


def _render_results_table() -> None:
    rows: List[str] = []
    folder_web = _html_escape(UI_STATE.get("folder_web") or "")
    for item in UI_STATE["results"]:
        name = _html_escape(item.get("name", "file"))
        gd_link = _html_escape(item.get("drive_link", ""))
        src = _html_escape(item.get("source_url", ""))
        rows.append(
            (
                "<tr>"
                f"<td class='cud-td-name'>{name}</td>"
                f"<td class='cud-td-link'><a class='cud-link' href='{folder_web}' target='_blank'>Folder</a></td>"
                f"<td class='cud-td-link'><a class='cud-link-src' href='{gd_link}' target='_blank'>Zasób</a></td>"
                f"<td class='cud-td-link'><a class='cud-link' href='{src}' target='_blank'>Źródło</a></td>"
                "</tr>"
            )
        )
    body = (
        "<div class='cud-help'>Brak wyników.</div>"
        if not rows
        else (
            "<table class='cud-table'>"
            "<thead><tr>"
            "<th class='cud-th-name'>Nazwa</th><th class='cud-th-link'>Dysk</th><th class='cud-th-link'>Zasób</th><th class='cud-th-link'>Źródło</th>"
            "</tr></thead>"
            f"<tbody>{''.join(rows)}</tbody></table>"
        )
    )
    RESULTS_PANEL.value = (
        "<div class='cud-results'><div class='cud-results-title big'>Wyniki</div>" + body + "</div>"
    )


def make_ui_progress_generic() -> Callable[[str], None]:
    throttle_sec = 1.0
    last = {"t": 0.0, "s": ""}

    def _cb(line: str) -> None:
        s = (line or "").rstrip()
        if not s:
            return
        if s == last["s"]:
            return
        now = time.time()
        if (now - last["t"]) < throttle_sec:
            return
        last["t"], last["s"] = now, s
        html = f"<pre class='cud-pre-oneline'>{_html_escape(s)}</pre>"
        _set_progress_panel(html)

    return _cb


def _ensure_drive_folder_with_link(
    gd: GoogleDriveManager,
    *,
    drive_id: str,
    session_folder: str,
) -> Tuple[str, str]:
    svc = gd.drive_service
    q = (
        f"name = '{session_folder.replace("'", "\\'")}' and '{drive_id}' in parents and trashed = false and mimeType = 'application/vnd.google-apps.folder'"
    )
    resp = svc.files().list(
        q=q,
        fields="files(id, name, webViewLink)",
        pageSize=1,
        supportsAllDrives=True,
        includeItemsFromAllDrives=True,
    ).execute()
    items = resp.get("files", []) or []
    if items:
        it = items[0]
        return it["id"], it.get("webViewLink", "")
    body = {
        "name": session_folder,
        "mimeType": "application/vnd.google-apps.folder",
        "parents": [drive_id],
    }
    created = (
        svc.files()
        .create(body=body, fields="id, webViewLink", supportsAllDrives=True)
        .execute()
    )
    return created["id"], created.get("webViewLink", "")


def _start_delete_scheduler() -> None:
    ds = UI_STATE.get("delete_scheduler")
    if ds.get("thread") is not None:
        return

    def _worker() -> None:
        while True:
            try:
                with ds["lock"]:
                    folder_id = ds.get("folder_id")
                    delete_at = float(ds.get("delete_at") or 0.0)
                if folder_id and delete_at:
                    now = time.time()
                    if now >= delete_at:
                        try:
                            UI_STATE["gdrive"].delete_folder_now(folder_id)  # type: ignore
                        except Exception:
                            pass
                        with ds["lock"]:
                            ds["folder_id"] = None
                            ds["delete_at"] = 0.0
                time.sleep(1.0)
            except Exception:
                time.sleep(1.0)

    t = threading.Thread(target=_worker, daemon=True)
    ds["thread"] = t
    t.start()


def _schedule_delete(folder_id: Optional[str], seconds: int) -> None:
    if not folder_id:
        return
    ds = UI_STATE["delete_scheduler"]
    with ds["lock"]:
        ds["folder_id"] = folder_id
        ds["delete_at"] = time.time() + max(1, int(seconds))
    _start_delete_scheduler()


pixeldrain_handler = PixelDrainDownloader()
mega_handler = MegaDownloader()
gdrive_handler = GoogleDriveManager()
UI_STATE.update({"gdrive": gdrive_handler, "pixeldrain": pixeldrain_handler, "mega": mega_handler})


def run_queue(
    *,
    links_text: str,
    drive_type: str,
    shared_drive_name: str,
    session_folder_name: str,
    delete_delay_seconds: int,
) -> None:
    progress_cb = make_ui_progress_generic()
    _set_progress_panel("<pre class='cud-pre-oneline'>Przygotowanie…</pre>")

    tasks = parse_links(links_text)
    if not tasks:
        _set_progress_panel("<div class='cud-help'>Wklej przynajmniej jeden link.</div>")
        return

    pxd_tmp = Path(BASE_TMP, "pixeldrain"); pxd_tmp.mkdir(parents=True, exist_ok=True)
    mega_tmp = Path(BASE_TMP, "mega"); mega_tmp.mkdir(parents=True, exist_ok=True)

    downloaded: List[Tuple[str, str]] = []

    for t in tasks:
        kind = t.get("type"); url = t.get("url", "")
        if kind not in ("pixeldrain", "mega_file", "mega_folder"):
            continue
        try:
            if kind == "pixeldrain":
                _set_progress_panel("<pre class='cud-pre-oneline'>[pixeldrain] Pobieranie…</pre>")
                path = pixeldrain_handler.download(url, dest_dir=str(pxd_tmp), progress_line=progress_cb)
                if path and Path(path).exists():
                    downloaded.append((path, url))
                    _set_progress_panel("<pre class='cud-pre-oneline'>[pixeldrain] Zakończono.</pre>")
            else:
                res = mega_handler.download(url, dest_dir=str(mega_tmp), progress_line=progress_cb)
                if isinstance(res, list):
                    for p in res:
                        if p and Path(p).exists():
                            downloaded.append((p, url))
                elif res and Path(res).exists():
                    downloaded.append((res, url))
        except Exception:
            continue

    is_shared = (str(drive_type).lower().strip() == "shared")
    drv_name = (shared_drive_name or "") if is_shared else "My Drive"
    sess_name = (session_folder_name.strip() or "ColabUniversalDownloader")

    cached = UI_STATE.get("drive_ctx") or {}
    folder_id: Optional[str] = None
    if (
        cached.get("is_shared") == is_shared
        and cached.get("drv_name") == drv_name
        and cached.get("session_folder") == sess_name
        and cached.get("folder_id")
    ):
        folder_id = cached.get("folder_id")
        UI_STATE["folder_web"] = cached.get("folder_web", "")
    else:
        try:
            root_parent_id: Optional[str] = gdrive_handler.get_drive_id(drive_name=drv_name, is_shared=is_shared)
        except Exception as ex:
            _set_progress_panel(f"<div class='cud-help'>Błąd pobierania ID Dysku: {_html_escape(str(ex))}</div>")
            return
        if not root_parent_id:
            _set_progress_panel("<div class='cud-help'>❌ Nie znaleziono dysku docelowego.</div>")
            return
        try:
            folder_id2, folder_web = _ensure_drive_folder_with_link(
                gdrive_handler,
                drive_id=root_parent_id,
                session_folder=sess_name,
            )
            UI_STATE["folder_web"] = folder_web or ""
            folder_id = folder_id2
            UI_STATE["drive_ctx"] = {
                "is_shared": is_shared,
                "drv_name": drv_name,
                "session_folder": sess_name,
                "folder_id": folder_id2,
                "folder_web": folder_web or "",
            }
        except Exception as ex:
            _set_progress_panel(f"<div class='cud-help'>Błąd tworzenia/odnajdywania folderu: {_html_escape(str(ex))}</div>")
            return

    _schedule_delete(folder_id, int(delete_delay_seconds or 0))

    for local_path, source_url in downloaded:
        name = os.path.basename(local_path)
        try:
            info = gdrive_handler.upload_and_share(local_path, parent_id=folder_id, progress_line=progress_cb)  # type: ignore
        except Exception:
            info = None
        drive_link = (info or {}).get("link") or ""
        UI_STATE["results"].append({
            "name": name,
            "drive_link": drive_link,
            "source_url": source_url,
        })
        _render_results_table()

    _set_progress_panel("")


HTML_UI = r"""
<style>
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;800&display=swap');
*{box-sizing:border-box}
.cud-wrap{font-family:'Inter',system-ui,-apple-system,Segoe UI,Roboto,sans-serif;background:#0f172a;color:#e2e8f0;border:1px solid #334155;border-radius:14px;padding:16px;max-width:880px;margin:10px auto}
.cud-title{font-weight:800;letter-spacing:.3px;font-size:20px;color:#93c5fd;text-align:center;margin:0 0 8px}
.cud-card{background:#0b1220;border:1px solid #334155;border-radius:12px;padding:12px;margin-top:10px}
.cud-grid-2{display:grid;grid-template-columns:1fr 1fr;gap:12px}
.cud-card .cud-grid-2{align-items:center}
.cud-card .cud-grid-2 > div{display:flex;flex-direction:column;justify-content:center}
.cud-grid-1{display:grid;grid-template-columns:1fr;gap:10px}
.cud-input,.cud-select,.cud-textarea{width:100%;background:#0b1220;color:#e2e8f0;border:1px solid #334155;border-radius:10px;padding:10px;font-size:13px}
#cud-folder,#cud-delay,#cud-shared-name{padding:14px 12px;font-size:14px;min-height:44px}
.cud-textarea{min-height:110px;white-space:pre-wrap;word-break:break-word}
.cud-btn{background:#60a5fa;color:#0f172a;font-weight:800;border:none;padding:12px 16px;border-radius:10px;cursor:pointer;transition:.15s;width:100%}
.cud-btn:hover{filter:brightness(1.1)}
.cud-radio{display:flex;gap:8px}
.cud-radio .opt{flex:1;text-align:center;padding:10px;border:1px solid #334155;border-radius:10px;cursor:pointer;user-select:none;height:44px;display:flex;align-items:center;justify-content:center}
.cud-radio .opt.active{background:#1e293b;border-color:#60a5fa;color:#93c5fd;font-weight:700}
.cud-help{font-size:11px;color:#94a3b8;margin-top:6px;opacity:.9}
.cud-card .cud-grid-2 > div > .cud-grid-2 > div > .cud-help{margin-bottom:6px}
.cud-pre{margin:10px 0 0;padding:10px;background:#0b1220;border:1px solid #334155;border-radius:8px;white-space:pre-wrap;overflow:auto}
.cud-pre-oneline{margin:10px 0 0;padding:10px;background:#0b1220;border:1px solid #334155;border-radius:8px;white-space:pre;overflow:hidden;font-family:ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, 'Liberation Mono', 'Courier New', monospace}
.cud-results{margin-top:10px;background:#0b1220;border:1px solid #334155;border-radius:12px;padding:12px}
.cud-results-title{font-weight:700;color:#93c5fd;margin-bottom:8px}
.cud-results-title.big{font-size:18px;letter-spacing:.2px}
.cud-table{width:100%;border-collapse:collapse;table-layout:fixed}
.cud-table th,.cud-table td{border-bottom:1px solid #1f2937;padding:8px;font-size:13px;vertical-align:middle;text-align:center}
.cud-th-name,.cud-td-name{width:60%;overflow:hidden;text-overflow:ellipsis;white-space:nowrap}
.cud-th-link,.cud-td-link{width:13%;text-align:center}
.cud-link{color:#93c5fd;font-weight:700;text-decoration:underline;font-size:16px;display:inline-block}
.cud-link-src{color:#34d399;font-weight:700;text-decoration:underline;font-size:16px;display:inline-block}
@media (max-width:900px){.cud-grid-2{grid-template-columns:1fr}}
</style>
<div class="cud-wrap">
  <div class="cud-title">ColabUniversalDownloader</div>
  <div class="cud-card">
    <label class="cud-help">Wklej linki: PixelDrain lub Mega (oddziel spacją lub nową linią)</label>
    <textarea id="cud-links" class="cud-textarea" placeholder="https://pixeldrain.com/u/...&#10;https://mega.nz/file/...#..."></textarea>
  </div>
  <div class="cud-card">
    <div class="cud-grid-2">
      <div>
        <div class="cud-help" style="margin-bottom:6px;">Dysk docelowy</div>
        <div class="cud-radio" id="cud-drive">
          <div class="opt active" data-val="my">Mój dysk</div>
          <div class="opt" data-val="shared">Dysk współdzielony</div>
        </div>
        <div id="row-shared" style="display:none; margin-top:8px;">
          <input id="cud-shared-name" class="cud-input" placeholder="Nazwa dysku współdzielonego" />
        </div>
      </div>
      <div>
        <div class="cud-grid-2">
          <div>
            <div class="cud-help">Folder sesji</div>
            <input id="cud-folder" class="cud-input" value="ColabUniversalDownloader" />
          </div>
          <div>
            <div class="cud-help">Auto-usunięcie (sekundy)</div>
            <input id="cud-delay" class="cud-input" type="number" min="0" step="60" value="600" />
          </div>
        </div>
      </div>
    </div>
    <button id="cud-start" class="cud-btn" style="margin-top:10px;">Start</button>
  </div>
</div>
<script>
(function(){
  const byId = (i) => document.getElementById(i);
  const qsa = (s) => Array.from(document.querySelectorAll(s));
  const isColab = !!(window.google && google.colab && google.colab.kernel && google.colab.kernel.invokeFunction);
  const state = { drive: 'my' };
  qsa('#cud-drive .opt').forEach(el => {
    el.addEventListener('click', () => {
      qsa('#cud-drive .opt').forEach(x => x.classList.remove('active'));
      el.classList.add('active');
      state.drive = el.dataset.val || 'my';
      byId('row-shared').style.display = state.drive === 'shared' ? 'block' : 'none';
    });
  });
  if (!isColab) {
    byId('cud-start').disabled = true;
  }
  byId('cud-start').addEventListener('click', () => {
    const payload = {
      links: byId('cud-links').value || '',
      drive: state.drive,
      sharedName: byId('cud-shared-name').value || '',
      folder: byId('cud-folder').value || 'ColabUniversalDownloader',
      delay: parseInt(byId('cud-delay').value || '600') || 0
    };
    try {
      if (isColab) {
        google.colab.kernel.invokeFunction('cud-handle-start-v2', [JSON.stringify(payload)], {});
      }
    } catch(e) { /* ignoruj poza Colab */ }
  });
})();
</script>
"""
HTML_UI = HTML_UI.replace("{{", "{").replace("}}", "}")
main_ui = widgets.HTML(HTML_UI)
container = widgets.VBox([main_ui, PROGRESS_PANEL, RESULTS_PANEL, HIDDEN_LOG])
UI_STATE["widget"] = container
display(container)


def _cb_start_v2(payload_json: str) -> None:
    try:
        data = json.loads(payload_json)
    except Exception:
        return
    run_queue(
        links_text=data.get("links") or "",
        drive_type=data.get("drive") or "my",
        shared_drive_name=data.get("sharedName") or "",
        session_folder_name=data.get("folder") or "ColabUniversalDownloader",
        delete_delay_seconds=int(data.get("delay") or 0),
    )

try:
    from google.colab import output as colab_output  # type: ignore
    colab_output.register_callback("cud-handle-start-v2", _cb_start_v2)
except Exception:
    pass
